<a href="https://colab.research.google.com/github/Beno71/humpback-whale-classification/blob/master/Colab_siamese_network_Cross_Entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os 
print(os.getcwd())
!ls

# siamese networks for whale classification

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.rendered_html { font-size: 18px; }</style>"))

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os, time, itertools
from skimage import io, transform
import skimage
import glob
from tqdm import tnrange, tqdm
from collections import Counter
from random import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import euclidean_distances
from IPython.display import clear_output

%matplotlib inline

# Load Data

In [5]:
# some prep steps
# for google colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
  
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive/')
  data_folder = "/content/drive/My Drive/Colab Notebooks/data/"
else:
  data_folder = "data/"


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#load .npz-file from folder



loader = np.load(data_folder+"humpback_300x100_gray_no_new.npz")
features = loader["features"]
labels = loader["labels"]

n_rows = labels.shape[0]



split_ratio = 0.8
data_size = 400
expansion_factor = 1

#take the sample from the most common classes
ar = np.array(Counter(labels).most_common())
count = 0
label_list = []
for idx, tup in enumerate(ar):
    label_list.append(tup[0])
    count += tup[1]
    if count > data_size:
        break
label_list

label_in_list=[x in label_list for x in labels]
labels = labels[label_in_list]
features = features[label_in_list]

def standardize(X):
    X = X.astype(np.float32)
    X = (X - np.mean(X, axis=(1,2), keepdims=True)) / np.std(X, axis=(1,2), keepdims=True)
    return X


features = standardize(features)
features = np.expand_dims(features, axis=3)

# not needed
#all_combinations_without_labels = np.array(list(itertools.combinations(range(data_size),2)))
#same_label_indices = labels[all_combinations_without_labels[:,0]] == labels[all_combinations_without_labels[:,1]]
#all_combinations = np.append(all_combinations_without_labels, same_label_indices.reshape(-1,1), axis=1)

In [7]:
features.shape

(423, 100, 300, 1)

In [0]:
def train_test_split_old(split_ratio):
    """
    Get unique combinations from [0..data_size].
    Shuffle same_label_pairs and dif_label_pairs individually.
    Sample the first split_ratio*num pairs of each class for training and 
    sample the the rest (1-split_ratio)*num pairs of each class for validation
    
    """
    same_label_pairs = np.random.permutation(all_combinations[same_label_indices])
    dif_label_pairs = np.random.permutation(all_combinations[~same_label_indices])
    
    num_pairs = int(1/2*expansion_factor*data_size)
    num_train = int(split_ratio*num_pairs)
   
    train_matchings = np.append(same_label_pairs[:num_train], dif_label_pairs[:num_train], axis = 0)
    val_matchings = np.append(same_label_pairs[num_train:num_pairs], dif_label_pairs[num_train:num_pairs], axis = 0)
    
    np.random.shuffle(train_matchings), np.random.shuffle(val_matchings)
    
    return train_matchings.astype(int), val_matchings.astype(int)

In [0]:
def train_test_split_one_hot(split_ratio):
    """
    Split data into train, validation and test set.
    Test set contains just pictures. 
    Validation and Train set are pairs of pictures (unique matches).
    """
    perm = np.random.permutation(range(data_size))
    train_idx = perm[:int(split_ratio*data_size)]
    test_idx = perm[int(split_ratio*data_size):]
    
    combinations = np.array(list(itertools.combinations(train_idx,2)))
    
    same_label_indices = labels[combinations[:,0]] == labels[combinations[:,1]]
    combinations = np.append(np.append(combinations, same_label_indices.reshape(-1,1), axis=1), ~same_label_indices.reshape(-1,1), axis=1)

    same_label_pairs = np.random.permutation(combinations[same_label_indices])
    dif_label_pairs = np.random.permutation(combinations[~same_label_indices])
    
    num_pairs = int(1/2*expansion_factor*data_size)
    num_train = int(split_ratio*num_pairs)
   
    train_matchings = np.append(same_label_pairs[:num_train], dif_label_pairs[:num_train], axis = 0)
    val_matchings = np.append(same_label_pairs[num_train:num_pairs], dif_label_pairs[num_train:num_pairs], axis = 0)
    
    np.random.shuffle(train_matchings), np.random.shuffle(val_matchings)
    return train_matchings.astype(int), val_matchings.astype(int), test_idx.astype(int)

In [0]:
def train_test_split(split_ratio):
    """
    Split data into train, validation and test set.
    Test set contains just pictures. 
    Validation and Train set are pairs of pictures (unique matches).
    """
    perm = np.random.permutation(range(data_size))
    train_idx = perm[:int(split_ratio*data_size)]
    test_idx = perm[int(split_ratio*data_size):]
    
    combinations = np.array(list(itertools.combinations(train_idx,2)))
    
    same_label_indices = labels[combinations[:,0]] == labels[combinations[:,1]]
    combinations = np.append(combinations, same_label_indices.reshape(-1,1), axis=1)

    same_label_pairs = np.random.permutation(combinations[same_label_indices])
    dif_label_pairs = np.random.permutation(combinations[~same_label_indices])
    
    num_pairs = int(1/2*expansion_factor*data_size)
    num_train = int(split_ratio*num_pairs)
   
    train_matchings = np.append(same_label_pairs[:num_train], dif_label_pairs[:num_train], axis = 0)
    val_matchings = np.append(same_label_pairs[num_train:num_pairs], dif_label_pairs[num_train:num_pairs], axis = 0)
    
    np.random.shuffle(train_matchings), np.random.shuffle(val_matchings)
    return train_matchings.astype(int), val_matchings.astype(int), test_idx.astype(int)

### Train Test Split

In [11]:
train_matchings, val_matchings, test_idx = train_test_split(split_ratio)
train_matchings.shape, val_matchings.shape, test_idx.shape


((320, 3), (80, 3), (80,))

## Helper functions for calculation of accuracy

In [12]:
np.unique(labels)

array([ 12,  18,  32,  58, 166, 241, 265])

In [0]:
def get_unique_N(iterable, N):
    """Yields (in order) the first N unique elements of iterable. 
    Might yield less if data too short."""
    seen = set()
    for e in iterable:
        if e in seen: # ES IST NIE IN SEEN :D
            continue
        seen.add(e)
        yield e
        if len(seen) == N:
            return

In [0]:
def get_k_nearest(distance_matrix):
    dm = distance_matrix.copy()
    for i in range(dm.shape[1]):    
        dm[i,i] += 100000 
    top5_nearest = np.empty((distance_matrix.shape[0], 5))
    for idx, line in enumerate(dm):
        sorted_indices = np.argsort(line)
        top5_nearest[idx,:] = np.fromiter(get_unique_N(labels[sorted_indices], 5), int)
    #top5_nearest = labels[top5_nearest.astype(int)]
    return top5_nearest.astype(int)

In [0]:
weights_standard = np.array([1, 0.8, 0.6, 0.4, 0.2])
weights_first = np.array([1,0,0,0,0])
weights_half = np.array([1,0.5,0.33,0.25,0.20])
def calculate_accuracy_score(outputs_1, outputs_2=None, weights=weights_standard):
    distance_matrix = euclidean_distances(outputs_1, outputs_1[test_idx])
    top5_nearest = get_k_nearest(distance_matrix)
    #print(top5_nearest)
    true_labels = np.repeat(np.array([labels]), 5, axis=0).T
    prediction_matrix = top5_nearest == true_labels
    #print(np.count_nonzero(prediction_matrix))
    for row in prediction_matrix:
        for i, elt in enumerate(row):
            if elt:
                row[i+1:] = 0
                break
    #print(np.count_nonzero(prediction_matrix),"/",labels.shape[0],"unique whales have correct labels in the top 5")
    scores_per_image = prediction_matrix@weights_half
    score = np.sum(scores_per_image)/scores_per_image.shape[0]
    return score

In [0]:
def batch_data(num_data, batch_size):
    """ Yield batches with indices until epoch is over.
    
    Parameters
    ----------
    num_data: int
        The number of samples in the dataset.
    batch_size: int
        The batch size used using training.

    Returns
    -------
    batch_ixs: np.array of ints with shape [batch_size,]
        Yields arrays of indices of size of the batch size until the epoch is over.
    """
    
    data_ixs = np.random.permutation(np.arange(num_data))
    ix = 0
    while ix + batch_size < num_data:
        batch_ixs = data_ixs[ix:ix+batch_size]
        ix += batch_size
        yield batch_ixs

In [17]:
np.unique(labels)

array([ 12,  18,  32,  58, 166, 241, 265])

# Check initializer of variables

In [0]:
class SiamN:
    
    def __init__(self, name, learning_rate=0.001, height=100, width=300, channels=1, margin=0.5):
        
        self.name = name
        self.dropout = tf.placeholder_with_default(0.0, shape=(), name="dropout")
        self.learning_rate = learning_rate
        self.weights =[]
        self.biases =[]
        self.margin = margin
        
        self.X1 = tf.placeholder(shape=[None, height, width, channels], dtype=tf.float32, name="data_1") 
        self.X2 = tf.placeholder(shape=[None, height, width, channels], dtype=tf.float32, name="data_2") 
        self.Y = tf.placeholder(shape=[None,], dtype=tf.float32, name="labels") 
        
        self.output1 = self.forward_pass(self.X1, reuse=False)
        self.output2 = self.forward_pass(self.X2, reuse=True)
        self.logits = self.head(self.output1, self.output2)
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(self.logits > 0, tf.cast(self.Y, tf.bool)),tf.float32))
        self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, labels=self.Y, name="cross_entropy_loss"))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
        
    
    def forward_pass(self, X, reuse=False):
        
        convkwargs = {'padding': 'same', 'activation_fn': tf.nn.relu, 'reuse': reuse, 'weights_initializer': tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)}
        poolkwargs = {'kernel_size': 2, 'stride': 2, 'padding': 'same'}
        
        with tf.variable_scope("conv1") as scope:
            x = tf.contrib.layers.conv2d(X, 64, kernel_size=9, stride=2, scope=scope, **convkwargs)
            #x = tf.contrib.layers.batch_norm(x, scope=scope, reuse=reuse)
            x = tf.contrib.layers.max_pool2d(x, **poolkwargs)
        with tf.variable_scope("conv2") as scope:
            x = tf.contrib.layers.conv2d(x, 64, kernel_size=3, stride=1, scope=scope, **convkwargs)
            #x = tf.contrib.layers.batch_norm(x, scope=scope, reuse=reuse)
            x = tf.contrib.layers.max_pool2d(x, **poolkwargs)
        with tf.variable_scope("conv3") as scope:
            x = tf.contrib.layers.conv2d(x, 128, kernel_size=3, stride=1, scope=scope, **convkwargs)
            #x = tf.contrib.layers.batch_norm(x, scope=scope, reuse=reuse)
            x = tf.contrib.layers.max_pool2d(x, **poolkwargs)
        with tf.variable_scope("conv4") as scope:
            x = tf.contrib.layers.conv2d(x, 256, kernel_size=1, stride=1, scope=scope, **convkwargs)
            #x = tf.contrib.layers.batch_norm(x, scope=scope, reuse=reuse)
            x = tf.contrib.layers.max_pool2d(x, **poolkwargs)
        
        x = tf.contrib.layers.flatten(x)
        x = tf.contrib.layers.fully_connected(x, 1000, activation_fn=tf.nn.relu, weights_initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32))
        #x = tf.contrib.layers.batch_norm(x)
        
        output = x
            
        return output
    
    def head(self, output1, output2): 
        x1 = tf.multiply(output1, output2)
        x2 = tf.add(output1, output2)
        x3 = tf.abs(output2 - output1)
        x4 = tf.square(x3)
        x = tf.concat([x1, x2, x3, x4], axis = 1)
        
        x = tf.reshape(x,[-1,4,x1.shape[1],1])
        x = tf.contrib.layers.conv2d(x, 32, kernel_size=[4,1], activation_fn=tf.nn.relu, padding='valid')
        x = tf.reshape(x, [-1,x1.shape[1], 32, 1])
        x = tf.contrib.layers.conv2d(x, 1, kernel_size=[1,32], padding='valid')
        
        x = tf.contrib.layers.flatten(x)
        #x = tf.contrib.layers.dropout(x)
        print(x.shape)
        x = tf.contrib.layers.fully_connected(x, 1)
        print(x.shape)
        return tf.reshape(x,[-1])

    
    def train(self, features, train_matchings, val_matchings, epochs=20, dropout=0.0, batch_size=512):

        train_losses = []
        val_losses = []
        acc_scores = []
        train_accs = []
        val_accs = []
        acc_scores.append(0)
        
        config = tf.ConfigProto()
        #config.gpu_options.allow_growth=True
        self.session = tf.Session(config=config)
        session = self.session
        
        #tf.set_random_seed(0)
        session.run(tf.global_variables_initializer())
        
        train_loss, train_acc = session.run([self.loss, self.accuracy], feed_dict={self.X1: features[train_matchings[:,0]], self.X2: features[train_matchings[:,1]], self.Y: train_matchings[:,2]})
        val_loss, val_acc = session.run([self.loss, self.accuracy], feed_dict={self.X1: features[val_matchings[:,0]], self.X2: features[val_matchings[:,1]], self.Y: val_matchings[:,2]})

        #output = session.run(self.output1, feed_dict={self.X1: features})
        #acc_score = calculate_accuracy_score(output)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        #acc_scores.append(int(acc_score*100))
        train_accs.append(int(train_acc*100))
        val_accs.append(int(val_acc*100))
        
        print(f"Epoch 0 train_loss: {train_losses[-1]} val_loss: {val_losses[-1]} acc_score: {acc_scores[-1]} train_acc: {train_accs[-1]} val_acc: {val_accs[-1]}")  
        
        
        for epoch in range(epochs):
            if (epoch+1) % 5 == 0:
                print(f"Epoch {epoch+1}/{epochs} train_loss: {train_losses[-1]} val_loss: {val_losses[-1]} acc_score: {acc_scores[-1]} train_acc: {train_accs[-1]} val_acc: {val_accs[-1]}")  
            for batch_ixs in batch_data(train_matchings.shape[0], batch_size):
                    _ = session.run( self.optimizer, feed_dict={self.X1: features[train_matchings[batch_ixs,0]], self.X2: features[train_matchings[batch_ixs,1]], self.Y: train_matchings[batch_ixs,2]})  
            
            #TODO: remove boilerplate code, define function to calc accs and errors with flag print=TRUE/FALSE
            train_loss, train_acc = session.run([self.loss, self.accuracy], feed_dict={self.X1: features[train_matchings[:,0]], self.X2: features[train_matchings[:,1]], self.Y: train_matchings[:,2]})
            val_loss, val_acc = session.run([self.loss, self.accuracy], feed_dict={self.X1: features[val_matchings[:,0]], self.X2: features[val_matchings[:,1]], self.Y: val_matchings[:,2]})
        
            #output = session.run(self.output1, feed_dict={self.X1: features})
            #acc_score = calculate_accuracy_score(output)

            train_losses.append(train_loss)
            val_losses.append(val_loss)
            #acc_scores.append(int(acc_score*100))
            train_accs.append(int(train_acc*100))
            val_accs.append(int(val_acc*100))

        
        
        self.hist={'train_loss': np.array(train_losses),
           'val_loss': np.array(val_losses), "epochs_trained": epoch}
        
        

In [0]:
#model1.session.close()

In [24]:
#You can change layer types and the number of neurons by changing the following variables.
t = time.time()
epochs = 400
batch_size = 256

tf.reset_default_graph()
model1 = SiamN("first_model", learning_rate = 0.0001, margin = 1)

model1.train(features, train_matchings, val_matchings, epochs, batch_size=batch_size)
print("Training finished in", time.time()-t,"s.")


#works: 
#head1layers and 5 
#head1&2layers and 200 (0.0002) : 97 but then 50
#"" ... and 500 (0.0002) : 98
#he initializer made it better
#"" ... and 1000(0.0001) : 100 after 70 epochs (9 out of 10 times its stuck at 50 50 though)

(?, 1000)
(?, 1)
Epoch 0 train_loss: 0.6934834122657776 val_loss: 0.6939243078231812 acc_score: 0 train_acc: 49 val_acc: 48
Epoch 5/400 train_loss: 0.6820300817489624 val_loss: 0.6970582008361816 acc_score: 0 train_acc: 53 val_acc: 47
Epoch 10/400 train_loss: 0.6598031520843506 val_loss: 0.6995150446891785 acc_score: 0 train_acc: 54 val_acc: 50
Epoch 15/400 train_loss: 0.6321255564689636 val_loss: 0.7071720957756042 acc_score: 0 train_acc: 64 val_acc: 47
Epoch 20/400 train_loss: 0.5903524160385132 val_loss: 0.7210813760757446 acc_score: 0 train_acc: 65 val_acc: 50
Epoch 25/400 train_loss: 0.5397167205810547 val_loss: 0.7563114166259766 acc_score: 0 train_acc: 70 val_acc: 50
Epoch 30/400 train_loss: 0.48033127188682556 val_loss: 0.7923809885978699 acc_score: 0 train_acc: 89 val_acc: 53
Epoch 35/400 train_loss: 0.43941086530685425 val_loss: 0.8537197113037109 acc_score: 0 train_acc: 95 val_acc: 57
Epoch 40/400 train_loss: 0.449535995721817 val_loss: 1.1093568801879883 acc_score: 0 train_

KeyboardInterrupt: ignored

In [0]:
test = tf.constant([[1,2,3], [1,2,3]])
test.shape.ndims

In [0]:
import pdb; pdb.pm()

> /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/variable_scope.py(861)_get_single_variable()
-> name, "".join(traceback.format_list(tb))))
(Pdb) model1.X1
*** NameError: name 'model1' is not defined
(Pdb) quit


In [0]:
scores_1, scores_2

In [0]:
calculate_accuracy_score(outputs)

NameError: ignored

In [0]:
score

In [0]:
idx = np.random.randint(3200)
pic_a = features[train_matchings[idx,0]]
pic_b = features[train_matchings[idx,1]]
print(f"same whales? {bool(train_matchings[idx,2])}")    
print("Index: ", idx)
distance = model1.session.run(model1.distance, feed_dict={model1.X_1: np.array([pic_a]), model1.X_2: np.array([pic_b]), model1.Y: np.array([train_matchings[idx,2]])})
print(f"distance: {distance}")
plt.figure(figsize=(10,5))
plt.imshow(pic_a[0], cmap="gray")
plt.figure(figsize=(10,5))
plt.imshow(pic_b[0], cmap="gray")

In [0]:

print("model 1")
plt.figure(figsize=(10,5))
plt.plot(model1.hist['train_loss'][5::], label="Training")
plt.plot(model1.hist['val_loss'][5::], label="Validation")

plt.xlabel("Epoch", fontsize=20)
plt.ylabel("Loss", fontsize=20)
plt.legend()
plt.show()

In [0]:
print(model1.hist["train_loss"])

# Experimental: tensorflow datasets

In [0]:
train_data = tf.data.Dataset.from_tensor_slices({"feature": train_data, "label": train_labels})
val_data = tf.data.Dataset.from_tensor_slices({"feature": val_data, "label": val_labels})
train_data

In [0]:
train_data.output_types

In [0]:
#build batches
batch_size = 500
train_data.shuffle(30000)
batches = dataset.batch(batch_size)

In [0]:


sess = tf.Session()
iterator = batches.make_one_shot_iterator()
next_element = iterator.get_next()
no_of_batches = int(np.ceil(labels.shape[0] / batch_size))
counter = 1
for i in tqdm(range(no_of_batches)):
    value = sess.run(next_element)
    print(value["feature"].shape)
    print(counter)
    counter+=1

sess.close()

In [0]:
# run this to check available gpu memory (i got 5gb)

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()